## Segmentation

In [1]:
# import some common libraries
import numpy as np
import cv2
import random

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Detectron
import detectron2

# Helper Library
import os
import json
from PIL import Image
import matplotlib
from matplotlib import pyplot as plt

1.7.1+cu110 True


In [11]:
torch.cuda.empty_cache()

##### Helper

In [3]:
def find_all_model_in_folder(folder_dir):
    for root, dirs, files in os.walk(folder_dir):
        for file in files:
            if file.endswith(".pth") and file.startswith("model"):
                path = os.path.join(root, file)
                path = path.replace("\\", "/")
                print(path)

#### Load Model

In [4]:
# your code goes here
find_all_model_in_folder('./results')

./results/faster-rcnn/05.04.2021, 22;28;07/COCO-Detection/faster_rcnn_R_50_C4_1x.yaml/model_final.pth
./results/faster-rcnn/05.05.2021, 04;58;13/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml/model_final.pth
./results/faster-rcnn/05.05.2021, 05;20;20/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml/model_final.pth
./results/faster-rcnn/05.22.2021, 13;06;41/COCO-Detection/faster_rcnn_R_50_C4_1x.yaml/model_final.pth
./results/faster-rcnn/05.22.2021, 13;19;25/COCO-Detection/faster_rcnn_R_50_C4_1x.yaml/model_final.pth


In [5]:
## Training
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
## Validation
from detectron2.engine import DefaultPredictor

from detectron2 import model_zoo


cfg_zoo = "faster_rcnn_R_50_C4_1x.yaml"
cfg_model_path = "./results/faster-rcnn/05.04.2021, 22;28;07/COCO-Detection/faster_rcnn_R_50_C4_1x.yaml/model_final.pth"

# Initialize predictor
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/" + cfg_zoo))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.WEIGHTS = cfg_model_path
predictor = DefaultPredictor(cfg)

#### Load Images

In [6]:
def get_dict_all_img(folder):
    images_dict = {}
    for filename in os.listdir(folder):
        filename_number = filename.split('_')[0]
        filename_type = filename.split('_')[1]
        if filename_number not in images_dict:
            images_dict[filename_number] = {}
        
        if filename_type.find("selfie") == -1:
            # e-ktp image
            images_dict[filename_number]['card'] = {
                'path': os.path.join(folder,filename),
                'bbox': None,
                'format': filename.split('.')[1]
            }
        else:
            images_dict[filename_number]['selfie'] = {
                'path': os.path.join(folder,filename),
                'bbox': None,
                'format': filename.split('.')[1]
            }
        
    return images_dict

In [7]:
# it should take a while since we put all the images into phython cache
train_dir = "./dataset/e-ktp/train/images"
val_dir = "./dataset/e-ktp/val/images"
train_dict = get_dict_all_img(train_dir)
val_dict = get_dict_all_img(val_dir)

images_dict = dict(train_dict, **val_dict)

#### Remove Background and Verification (Detect)
We will try to use a GrabCut and probably there's some images that heavily blurred and the algorithm will not taking the actual foreground of the images. But this will be a room of improvement in this project

In [8]:
# removing background on selfie images
## TODO

In [13]:
# Somehow adding this line make the predictor of the bounding boxes not error
# Without this lines, the predictor on cell below will got exploded by memory leak
im = cv2.imread(images_dict['7']["selfie"]["path"])
outputs = predictor(im)
images_dict['7']["selfie"]["bbox"] = outputs["instances"].to("cpu").pred_boxes

d:\project\skripsi\libs\detectron2\detectron2\modeling\roi_heads\fast_rcnn.py:154: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()


In [14]:
# localize all e-ktp position
for image_key in images_dict:
    # Localize selfie image
    im = cv2.imread(images_dict[image_key]["selfie"]["path"])
    outputs = predictor(im)
    images_dict[image_key]["selfie"]["bbox"] = outputs["instances"].to("cpu").pred_boxes

    # Localize card image
    im_ = cv2.imread(images_dict[image_key]["card"]["path"])
    outputs_ = predictor(im_)
    images_dict[image_key]["card"]["bbox"] = outputs_["instances"].to("cpu").pred_boxes

#### Crop

In [15]:
# Crop function
def get_cropped_imgs_bbox(img_path, bboxes):
    images = []
    bbox_arr = bboxes.tensor.cpu().numpy()
    im = cv2.imread(img_path)
    
    for bbox in bbox_arr:
        x0 = np.around(bbox[0]).astype(int) 
        y0 = np.around(bbox[1]).astype(int)
        x1 = np.around(bbox[2]).astype(int)
        y1 = np.around(bbox[3]).astype(int)
        
        images.append(im[y0:y1,x0:x1])
    
    return images

In [16]:
# Reset crop folder
crop_path = "./results/cropped/e-ktp"

# Delete previous cropped image
for filename in os.listdir(crop_path):
    file_path = os.path.join(crop_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [17]:
from PIL import Image

for image_key in images_dict:
    # Save cropped selfie results
    img_selfie_obj = images_dict[image_key]["selfie"]
    cropped_selfie_images = get_cropped_imgs_bbox(img_selfie_obj["path"], img_selfie_obj["bbox"])
    for idx, cropped_selfie_image in enumerate(cropped_selfie_images):
        imageRGB = cv2.cvtColor(cropped_selfie_image, cv2.COLOR_BGR2RGB)
        im = Image.fromarray(imageRGB)
        im.save(crop_path+"/"+image_key+"_selfie_crop_"+str(idx)+"."+img_selfie_obj["format"])
    
    # Save cropped card results
    img_card_obj = images_dict[image_key]["card"]
    cropped_card_images = get_cropped_imgs_bbox(img_card_obj["path"], img_card_obj["bbox"])
    for idx, cropped_card_image in enumerate(cropped_card_images):
        imageRGB = cv2.cvtColor(cropped_card_image, cv2.COLOR_BGR2RGB)
        im = Image.fromarray(imageRGB)
        im.save(crop_path+"/"+image_key+"_card_crop_"+str(idx)+"."+img_card_obj["format"])
        